# Por que Python?

- **Linguagem mais usada em Data Science**
- Fácil de aprender: sintaxe simples
- Open source
- Grande número de bibliotecas disponíveis
- Desvantagem: desempenho computacional por ser interpretada

# Breve Intro a Python

In [ ]:
# Variaveis sao dinamicamente tipadas
nome = "João"
idade = 45
peso = 85.3
ehBrasileiro = True

In [ ]:
print("nome:" + str(type(nome)))
print("idade:" + str(type(idade)))
print("peso:" + str(type(peso)))
print("ehBrasileiro:" + str(type(ehBrasileiro)))

nome:<class 'str'>
idade:<class 'int'>
peso:<class 'float'>
ehBrasileiro:<class 'bool'>


In [ ]:
# Operações com strings
comp = "João" > "Pedro"
print(comp)

comp = "João" == "Pedro"
print(comp)

sub = "Jose" in "Josevaldo"
print(sub)

parts = "idade,peso,altura".split(",")
print(parts)

False
False
True
['idade', 'peso', 'altura']


In [ ]:
# Operadores lógicos
comp = 4 < 5 and 9 > 10
print(comp)
comp = 4 < 5 or 9 > 10
print(comp)
comp = 4 < 5 and not (9 > 10)
print(comp)

False
True
True


In [ ]:
# commando print
x = 3
y = 4
print(x, "elevado a", y, "é", x**y)
print(str(x) + " elevado a " + str(y) + " é " + str(x**y))

3 elevado a 4 é 81
3 elevado a 4 é 81


In [ ]:
# Testes condicionais
if idade > 65:
    # Indentacao para definir o nivel das instrucoes
    print(nome, "é um candidato a se aposentar")
    print("Erro")
else:
    print(nome, "tem de trabalhar mais um pouco")

João tem de trabalhar mais um pouco


In [ ]:
# Listas possuem tipos diferentes e podem ser modificadas
dados = [nome, idade, peso, 5]
print(dados)

# Tuplas possuem tipos diferentes mas não podem ser modificadas
ponto = (1,3,-1)
print(ponto)

# Dicionarios mapeiam uma chave a um valor
dic = {'nome':nome, 'idade':idade, 'peso':peso, 'end': 'Av. Jornalista Anibal Fernandes, Centro de Informática'}
print(dic)
print(f"Nome: {dic['nome']:s}")

['João', 45, 85.3, 5]
(1, 3, -1)
{'nome': 'João', 'idade': 45, 'peso': 85.3, 'end': 'Av. Jornalista Anibal Fernandes, Centro de Informática'}
Nome: João


In [ ]:
# Operações em listas
dados.append("Recife")
print(dados)

dados.remove("Recife")
print(dados)

print(dados[1])

print("Tamanho =", len(dados))

['João', 45, 85.3, 5, 'Recife']
['João', 45, 85.3, 5]
45
Tamanho = 4


In [ ]:
# Operações em dicionários
dic["cidade"] = "Recife"
print(dic)
dic["cidade"] = "Salvador"
print(dic)

print(dic.get("cidade"))

dic.clear()
print(len(dic))

{'cidade': 'Recife'}
{'cidade': 'Salvador'}
Salvador
0


In [ ]:
# Repeticoes podem ser feitas de duas formas
# 1 - usando lacos while
vezes = 1
while vezes <= 5:
    print("Já passei por aqui {0} vez(es)".format(vezes))
    vezes += 1

Já passei por aqui 1 vez(es)
Já passei por aqui 2 vez(es)
Já passei por aqui 3 vez(es)
Já passei por aqui 4 vez(es)
Já passei por aqui 5 vez(es)


In [ ]:
# 2 - enumerando os elementos de uma lista (objeto iteravel)
for vezes in [1, 2, 3, 4, 5]:
    print("Já passei por aqui {0} vez(es)".format(vezes))
print("====")
# Ou ainda
for vezes in range(1,6,1):
    print("Já passei por aqui {0} vez(es)".format(vezes))

Já passei por aqui 1 vez(es)
Já passei por aqui 2 vez(es)
Já passei por aqui 3 vez(es)
Já passei por aqui 4 vez(es)
Já passei por aqui 5 vez(es)
====
Já passei por aqui 1 vez(es)
Já passei por aqui 2 vez(es)
Já passei por aqui 3 vez(es)
Já passei por aqui 4 vez(es)
Já passei por aqui 5 vez(es)


In [ ]:
for dado in dados:
    print(dado)

João
45
85.3
5


In [ ]:
for i in range(0,len(dados),1):
    print(dados[i])

João
45
85.3
5


In [ ]:
#Slicing [>=,<]
print(dados[1:3])

[45, 85.3]


In [ ]:
# Funcoes em Python
def quadrado(x):
    return x**2

print(quadrado(3))

# Funcoes mais simples podem ser definidas como funcoes lambda
f = lambda x: x**2
print(f(3))

9
9


# Processamento Orientado à Coluna

In [10]:
import sklearn
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)

In [11]:
print(data["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

# Frameworks

## 1- Pandas

Framework para manipulação de dados tabulares em memória

* **Pros**: API extremamente extensiva (inspirada nos dataframes nativos de R)
* **Cons**: single-thread

Operações podem ser feitas "de uma vez só" numa coluna inteira ao invés de ir linha por linha. `pandas` implementa diversas otimizações por debaixo dos panos, de forma que operações como adicionar uma constante para todos valores de uma coluna é feita de forma *quasi*-simultânea

In [5]:
import pandas as pd

In [12]:
# Construção de data frames
df = pd.DataFrame(data["data"], columns=data["feature_names"])
df.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25


In [ ]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [ ]:
df.dtypes

MedInc        float64
HouseAge      float64
AveRooms      float64
AveBedrms     float64
Population    float64
AveOccup      float64
Latitude      float64
Longitude     float64
dtype: object

In [ ]:
# Selecionando uma coluna
print(df['Population'].head())

0     322.0
1    2401.0
2     496.0
3     558.0
4     565.0
Name: Population, dtype: float64


In [ ]:
# Selecionando múltiplas colunas
print(df[['Latitude','Longitude']].head())

   Latitude  Longitude
0     37.88    -122.23
1     37.86    -122.22
2     37.85    -122.24
3     37.85    -122.25
4     37.85    -122.25


In [ ]:
# Selecionando linhas com slice
df[2:5]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [ ]:
# Seleção de linhas e colunas: loc
print(df.loc[10:15,['Population','AveRooms']])

    Population  AveRooms
10       910.0  5.477612
11      1504.0  4.772480
12      1098.0  5.322650
13       345.0  4.000000
14      1212.0  4.262903
15       697.0  4.242424


In [ ]:
# Seleção de linhas e colunas: iloc
print(df.iloc[10:15,[4,2]])

    Population  AveRooms
10       910.0  5.477612
11      1504.0  4.772480
12      1098.0  5.322650
13       345.0  4.000000
14      1212.0  4.262903


In [ ]:
# Seleção de dados com expressões booleanas
df[(df['Population'] > 1000) & (df.AveBedrms > 1)].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26
11,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26
12,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26
14,1.9167,52.0,4.262903,1.009677,1212.0,1.954839,37.85,-122.26


# **Índice**

In [ ]:
df1 = df[(df.Population > 1000) & (df.AveBedrms > 1)]
df1.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26
11,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26
12,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26
14,1.9167,52.0,4.262903,1.009677,1212.0,1.954839,37.85,-122.26


In [ ]:
print(df1.loc[7:11,['MedInc','HouseAge']])

    MedInc  HouseAge
7   3.1200      52.0
8   2.0804      42.0
11  3.2705      52.0


In [ ]:
df1.reset_index()

,index,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25
1,8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26
2,11,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26
3,12,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26
4,14,1.9167,52.0,4.262903,1.009677,1212.0,1.954839,37.85,-122.26
...,...,...,...,...,...,...,...,...,...
10214,20632,3.1250,15.0,6.023377,1.080519,1047.0,2.719481,39.26,-121.45
10215,20633,2.5495,27.0,5.445026,1.078534,1082.0,2.832461,39.19,-121.53
10216,20634,3.7125,28.0,6.779070,1.148256,1041.0,3.026163,39.27,-121.56
10217,20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22


In [ ]:
df1.reset_index(inplace=True,drop = True)
df1.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25
1,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26
2,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26
3,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26
4,1.9167,52.0,4.262903,1.009677,1212.0,1.954839,37.85,-122.26


In [ ]:
print(df1.loc[0:1,['MedInc','HouseAge']])

   MedInc  HouseAge
0  3.1200      52.0
1  2.0804      42.0


In [ ]:
print(df1.iloc[0:2,[0,1]])

   MedInc  HouseAge
0  3.1200      52.0
1  2.0804      42.0


<img src="https://raw.githubusercontent.com/ProfLuciano/intro_cd/130b92d33280d3ae7e2e1f34e435b21f8e3a8025/notebooks/pandas_selection.png">

In [ ]:
%%time
# Calculando média de número de quartos usando orientação à linha
accumulator = 0
for record in data["data"]:
    accumulator += record[2]
accumulator /= len(data["data"])
print(f"The mean of mean number of bedrooms in california is {accumulator:.3f}")

The mean of mean number of bedrooms in california is 5.429
CPU times: user 11.5 ms, sys: 766 µs, total: 12.3 ms
Wall time: 17.6 ms


In [ ]:
%%time
# Calculando média usando orientada à coluna
mean = df.AveRooms.sum() / len(df)
print("The mean of mean number of bedrooms in california is {accumulator:.3f}")

The mean of mean number of bedrooms in california is {accumulator:.3f}
CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 2.52 ms


In [ ]:
%%time
# Forma mais simples
mean = df.AveRooms.mean()
print("The mean of mean number of bedrooms in california is {mean:.3f}")

The mean of mean number of bedrooms in california is {mean:.3f}
CPU times: user 1.44 ms, sys: 0 ns, total: 1.44 ms
Wall time: 1.38 ms


In [ ]:
# Função apply: aplica uma função a uma coluna inteira
from math import log
df["HouseAgeLog"] = df.HouseAge.apply(log)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,HouseAgeLog
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,3.713572
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.044522
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.951244
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.951244
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.951244


Exercícios:

1. Qual o centróide geográfico do dataset? (uma linha!)
2. Quantos quarteirões existem com idade média da casa abaixo de 10 anos e com número de habitantes médio abaixo de 3 pessoas?

In [ ]:
df[['Latitude','Longitude']].mean()

Latitude      35.631861
Longitude   -119.569704
dtype: float64

In [ ]:
len(df[(df['HouseAge'] < 10) & (df.AveOccup < 3)])

771

## 2 - Dask

> Dask is a flexible library for parallel computing in Python.

`dask` é uma alternativa multi-core e/ou distribuída para pandas.
* **Pros** : multi-core, cluster mode, escala
* **Cons**: API limitada, “pensar distribuído”, nem toda operação é trivialmente paralelizada

`dask` segue um modelo *lazy* computacional, então a operação não é de fato executada até que seja chamado `.compute()` explicitamente, retornando um dataframe `pandas` em memória.

In [1]:
!pip install "dask[dataframe]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import dask.dataframe as dd
from multiprocessing import cpu_count


In [13]:
ddf = dd.from_pandas(df, npartitions= cpu_count())

In [14]:
%%time
mean = ddf["AveRooms"].mean().compute()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

The mean of mean number of bedrooms in california is 5.429
CPU times: user 12.5 ms, sys: 1.7 ms, total: 14.2 ms
Wall time: 19.3 ms


Quando usar `dask` ao invés de `pandas`?

1. Dataset não couber em memória;
2. `Pandas` estiver lento e se deseja utilizar todos os cores da máquina para paralelizar o processamento;
3. Dataset estiver quebrado em inúmeros arquivos - relacionado a (1)

## 3 - Polars

In [17]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import polars as pl
import pandas as pd

In [20]:
df_pl = pl.from_pandas(df)

In [22]:
%%time
mean = df_pl["AveRooms"].mean()
print(f"The mean of mean number of bedrooms in california is {mean:.3f}")

The mean of mean number of bedrooms in california is 5.429
CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 2.25 ms


# Referências

* Docs
    * https://pandas.pydata.org/pandas-docs/stable/
    * http://docs.dask.org/en/latest/


* Designing Data-Intensive Applications
    * https://dataintensive.net 
    * https://www.amazon.com/Designing-Data-Intensive-Applications-Reliable-Maintainable/dp/1449373321


* Extensões do Jupyter notebook: 
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions.html
    * https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html
    * https://towardsdatascience.com/jupyter-notebook-extensions-517fa69d2231
    